In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation, BaseScheduler
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, Point, MultiPoint, box
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp
from numpy.random import rand, seed
import folium
import random 
from haversine import haversine
import plotly.express as px
from plotly.subplots import make_subplots
import base64
from io import BytesIO
from IPython.display import IFrame
from IPython.display import display, Javascript
from IPython.core.display import HTML
import copy
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
class ConstructionSite(Agent): 
    def __init__(self, unique_id, model, buildingType, coords, 
                 inA10, waterbound, start_year, end_year, multiplier):
        super().__init__(unique_id, model)
        self.buildingType = buildingType # A, B, or C
        self.coords = coords 
        self.inA10 = inA10 # True or False 
        self.waterbound = waterbound # True or False
        self.start_year = start_year
        self.end_year = end_year
        self.multiplier = multiplier
        self.materials_request = {}
        self.materials_request_forHubs = {}
        self.materials_request_forSuppliers = {}
        self.materials_received = {}
        self.material_composition_df = None
        
        if self.model.hub_network == 'decentralized': 
            self.nearestHub_id = None
            self.nearestHub_dist = None
        self.nearestMacroHub_id = None
        self.nearestMacroHub_dist = None
        
        self.filter_material_composition_df()
        self.calc_materials_required()
        if self.model.modularity_type == 'full': 
            self.add_materials_required_modular_core()
            
    def filter_material_composition_df(self): 
        '''make df of materials required, based on building, biobased, and modular type'''
        b = self.model.build_info.copy()
        b = b[(b.buildingType == self.buildingType) & (b.biobased_type == self.model.biobased_type)] 
        if self.model.modularity_type == 'none': 
            b = b[b.material != 'modules']
        else: # self.model.modularity_type == 'full': 
            b = b[(b.structural_type == 'foundation') | (b.material == 'modules')]
        b = b[~b.tons.isna()]
        b.tons = b.tons * self.multiplier
        self.material_composition_df = b
    
    def calc_materials_required(self): 
        '''make dictionaries required to record materials required and received on construction site'''
        self.materials_required = {}
        self.materials_received = {}
        self.material_composition_df.apply(lambda row: self._add_materials_required(row), axis=1)
        
    def add_materials_required_modular_core(self): 
        b = self.model.build_info.copy()
        b = b[(b.buildingType == self.buildingType) & 
              (b.biobased_type == self.model.biobased_type) & 
              (b.structural_type == 'structural')] 
        b = b[~b.tons.isna()]
        b.tons = b.tons * 0.2
        b.apply(lambda row: self._add_materials_required(row), axis=1)
        
        self.material_composition_df = pd.concat([self.material_composition_df, b])
        
    def _add_materials_required(self, row): 
        strucType = row.structural_type
        mat = row.material
        amount = row.tons

        self.materials_required[strucType] = self.materials_required.get(strucType, {})
        self.materials_required[strucType][mat] = self.materials_required[strucType].get(mat, 0) + amount

        self.materials_received[strucType] = self.materials_received.get(strucType, {})
        self.materials_received[strucType][mat] = 0
                
    def step(self): 
        if self.start_year <= self.model.year <= self.end_year + 1: 
            self.make_materials_request_dicts()
            self.request_materials()
        
    def make_materials_request_dicts(self): 
        self.materials_request_forHubs = {}
        self.materials_request_forSuppliers = {}
        self.materials_request_all = {}
        
        for strucType, mat_amounts in self.materials_required.items(): 
            self.materials_request_forHubs[strucType] = {}
            self.materials_request_forSuppliers[strucType] = {}
            self.materials_request_all[strucType] = {}
            
            for mat, amount in mat_amounts.items(): 
                if mat in self.model.materials_forHubs: 
                    self.materials_request_forHubs[strucType][mat] = amount
                elif mat in self.model.materials_forSuppliers: 
                    self.materials_request_forSuppliers[strucType][mat] = amount
                else: 
                    print('ERROR: mat not in list for hubs or suppliers')
                self.materials_request_all[strucType][mat] = amount
                
    def request_materials(self):
        '''make materials_request that will be received by supplier / macro / micro hub'''        
        materials_request_forHubs = self.materials_request_forHubs
        materials_request_forSuppliers = self.materials_request_forSuppliers
        materials_request_all = self.materials_request_all
        
        for strucType, mat_amounts in self.materials_required.items():             
            for mat, amount in mat_amounts.items(): 
                mat_required = amount 
                mat_received = self.materials_received[strucType][mat] 
                mat_stillNeeded = mat_required - mat_received
                
                mat_request = mat_required / (self.end_year - self.start_year + 1)
                mat_request = mat_request if mat_request < mat_stillNeeded else mat_stillNeeded
                if self.model.year > self.end_year: 
                    mat_request = 0
                                
                materials_request_all[strucType][mat] = mat_request
                if mat in self.model.materials_forHubs: 
                    materials_request_forHubs[strucType][mat] = mat_request
                elif mat in self.model.materials_forSuppliers: 
                    materials_request_forSuppliers[strucType][mat] = mat_request
                else: 
                    print(f'error: {mat} has not been assigned to hubs or suppliers')
                    
        if self.model.year > self.end_year: 
            materials_request_forHubs = {}
            materials_request_forSuppliers = {}
            materials_request_all = {}
                        
        self.materials_request_forSuppliers = materials_request_forSuppliers
        self.materials_request_forHubs = materials_request_forHubs
        self.materials_request_all = materials_request_all
            
        
class Hub(Agent):
    def __init__(self, unique_id, model, hubType, coords, inA10, waterbound):
        super().__init__(unique_id, model)
        self.hubType = hubType # macro or micro 
        self.coords = coords
        self.inA10 = inA10 # True of False
        self.waterbound = waterbound # True of False
        self.nearestMacroHub_id = None
        self.nearestMacroHub_dist = None
        
        self.materials_toSend = {}
        self.nTrips = {}
        self.materials_request = {}
        self.materials_request_forSuppliers = {}
        self.materials_received = dict.fromkeys(self.model.materials_list, 0)
        
        self.suppliers = {}
        self.clients = {}
        self.trucks_toSite = []
        self.vehicles_toSupplier = []
        self.demolition_site_ids = []
        self.supplier_ids = []
        self.client_ids = []
                                
    def step(self):
        self.find_clients()
        if self.clients: # if hub has clients: 
            # print(f'{self.hubType} hub {self.unique_id} is calculating materials to send... ')
            self.calc_materials_toSend() # to each site / microHub
            self.check_if_materials_toSend_hasAmounts()
            if not self.materials_toSend_hasAmounts: 
                self.materials_request = {}
            
            elif self.materials_toSend_hasAmounts: # if hub has materials to send: 
                # print(f'{self.hubType} hub {self.unique_id} is making materials request ... ')
                self.make_materials_request() # for suppliers / demSites / macroHubs 

                if self.hubType != 'macro_assembly': 
                    # print(f'{self.hubType} hub {self.unique_id} is triaging materials request ... ')
                    self.triage_materials_request()

                    if self.model.circularity_type != 'none' and self.hubType == 'macro': 
                        # print(f'{self.hubType} hub {self.unique_id} is collecting materials from dem sites ... ')
                        self.simplify_matRequest_forDemolitionSites()
                        self.collect_materials_fromDemolitionSites()

                else: # hubType == 'macro_assembly'
                    # print(f'{self.hubType} hub {self.unique_id} is making materials request for module supplier ... ')
                    self.materials_request_forModuleSupplier() 

                if self.hubType != 'micro': # hubType = 'macro' or 'macro_assembly' 
                    # print(f'{self.hubType} hub {self.unique_id} is collecting materials from supplier ... ')
                    self.find_suppliers()
                    self.collect_materials_fromSupplier() # collect materials from suppliers 

                # print(f'{self.hubType} hub {self.unique_id} is sending materials to client ... ')
                self.send_materials_toClient() # send materials to site / microhub
        
    def find_clients(self): 
        '''make dictionary of clients. 
            - For macrohubs, clients are nearby construction sites and micro hubs. 
            - For microhubs, clients are nearby construction sites. 
            - for macro assembly hubs, clients are all construction sites and microhubs (if hub_network is decentralized).
        self.clients = {id: {'agent': agentObject, 'distance': 14312}, ... etc}'''
        
        self.clients = {}
        
        if self.model.hub_network == 'decentralized': 
            if self.hubType == 'macro': 
                microHubs = [hub for hub in self.model.hubs if hub.hubType == 'micro']
                microHubs = [hub for hub in microHubs if hub.nearestMacroHub_id == self.unique_id]
                sites = [s for s in self.model.construction_sites if s.nearestHub_id == self.unique_id]
                clients = microHubs + sites # sometimes sends directly to sites if no microhub is closeby
                for client in clients: 
                    self.clients[client.unique_id] = {'agent': client, 'distance': client.nearestMacroHub_dist}
            
            elif self.hubType == 'macro_assembly': 
                microHubs = [hub for hub in self.model.hubs if hub.hubType == 'micro']
                clients = microHubs
                for client in clients: 
                    od = self.model.od_matrix_h2h
                    distance = od[(od[:, 0] == self.unique_id) & (od[:, 1] == client.unique_id)][0][2]
                    self.clients[client.unique_id] = {'agent': client, 'distance': distance}
            
            elif self.hubType == 'micro': 
                sites = self.model.construction_sites
                clients = [site for site in sites if site.nearestHub_id == self.unique_id]
                for client in clients: 
                    self.clients[client.unique_id] = {'agent': client, 'distance': client.nearestHub_dist}
        
        elif self.model.hub_network == 'centralized':
            if self.hubType == 'macro': 
                sites = self.model.construction_sites
                clients = [site for site in sites if site.nearestMacroHub_id == self.unique_id]
                for client in clients: 
                    self.clients[client.unique_id] = {'agent': client, 'distance': client.nearestMacroHub_dist}    
            else: # if self.hubType == 'macro_assembly'
                clients = self.model.construction_sites
                for client in clients: 
                    od = self.model.od_matrix_h2c
                    distance = od[(od[:, 0] == self.unique_id) & (od[:, 1] == client.unique_id)][0][2]
                    self.clients[client.unique_id] = {'agent': client, 'distance': distance}
                                    
    def calc_materials_toSend(self): 
        '''check which clients still need materials and make materials_toSend dictionary
        self.clients = microhubs or construction sites 
        self.materials_toSend = {site_id: {'foundation': {'timber': 123, ... }, ... }'''
        self.materials_toSend = {}
        matList = []
               
        for client_id in self.clients.keys():
            materials_request, sum_values = self._get_materials_request(client_id)
            
            if sum_values > 0: # if client is requesting anything: 
                matList = self._make_materials_list(materials_request)
                materials_toSend = self._make_materials_toSend_dict(client_id, matList, materials_request)
        
        self.materials_list = matList
        self.materials_toSend = self._remove_zeros(self.materials_toSend)
                
    def _get_materials_request(self, client_id): 
        client = self.clients[client_id]['agent']
        
        if isinstance(client, Hub): 
            materials_request = client.materials_request.copy()
        elif isinstance(client, ConstructionSite): 
            materials_request = client.materials_request_forHubs.copy()
        else: 
            print('somethings wrong here: couldnt get materials request')
        
        sum_values = sum(sum(request.values()) for request in materials_request.values())
        return materials_request, sum_values
    
    def _make_materials_list(self, materials_request): 
        matList = []
        for strucType, mat_amounts in materials_request.items(): 
            for mat, amount in mat_amounts.items(): 
                matList.append(mat)
                
        matList = list(set(matList))
        if self.hubType == 'macro_assembly': 
            matList = ['modules']
            
        return matList
    
    def _make_materials_toSend_dict(self, client_id, matList, materials_request): 
        
        self.materials_toSend[client_id] = {
            strucType: {
                mat: 0 for mat in matList 
            } for strucType in materials_request.keys()
        }
        for strucType, mat_amounts in materials_request.items(): 
            for mat, amount in mat_amounts.items(): 
                if mat in matList: 
                    self.materials_toSend[client_id][strucType][mat] += amount 
    
    def _remove_zeros(self, materials_toSend): 
        materials_toSend_noZeros = {}
        for client_id, toSend_dict in materials_toSend.items():
            materials_toSend_noZeros[client_id] = {}
            for strucType, mat_amounts in toSend_dict.items(): 
                materials_toSend_noZeros[client_id][strucType] = {}
                for mat, amount in mat_amounts.items(): 
                    if amount == 0: 
                        continue
                    materials_toSend_noZeros[client_id][strucType][mat] = amount
        return materials_toSend_noZeros
        
    def check_if_materials_toSend_hasAmounts(self): 
        hasAmounts = False
        for site_id, mat_request in self.materials_toSend.items(): 
            for strucType, mat_amounts in mat_request.items(): 
                for mat, amount in mat_amounts.items(): 
                    if amount > 0: 
                        hasAmounts = True
                        break
        self.materials_toSend_hasAmounts = hasAmounts
                            
    def make_materials_request(self):
        '''self.materials_request = {'foundation': {'timber': 123, ... }, ... }, ... }'''
                              
        materials_request = {}
        for site_id, mat_request in self.materials_toSend.items():
            for strucType, mat_amounts in mat_request.items(): 
                materials_request[strucType] = materials_request.get(strucType, {})
                for mat, amount in mat_amounts.items(): 
                    materials_request[strucType][mat] = materials_request[strucType].get(mat, 0) + amount 
        
        self.materials_request = materials_request
        
        # if self.unique_id == 373: 
        #     print(f'macro hub {self.unique_id} has the following mat request: \n{self.materials_request}')

                
    def materials_request_forModuleSupplier(self): 
        '''this function only runs for the macro_assembly hub. 
        it simplifies the materials_request into materials_request for suppliers, 
        which doesnt have information on structural type'''
        matReqSupplier = {} 
        for strucType, mat_amounts in self.materials_request.items(): 
            matReqSupplier[strucType] = {}
            for mat, amount in mat_amounts.items(): 
                matReqSupplier[strucType][mat] = matReqSupplier[strucType].get(mat, 0) + amount
        self.materials_request_forSuppliers = matReqSupplier
        
    def triage_materials_request(self):
        '''separate self.materials_request into two parts, 
        one for demolition sites and one for suppliers'''
        
        strucTypes_suppliers, strucTypes_demSites = self._pick_strucTypes_forTriage()
        
        self.materials_request_forDemSites = self._make_matRequest_triaged(strucTypes_demSites)
        self.materials_request_forSuppliers = self._make_matRequest_triaged(strucTypes_suppliers)
        
        # if self.unique_id == 373:
        #     print(f'supplier strucTypes: {strucTypes_suppliers}; demsite strucTypes: {strucTypes_demSites}')
        #     print(f'mat request for demsites: {self.materials_request_forDemSites}')
        #     print(f'mat request for supplers: {self.materials_request_forSuppliers}')
            
    def _pick_strucTypes_forTriage(self): 
        '''pick strucType for demSite and suppliers'''
        strucTypes_forCircParam_dict = {
            'none': [], 
            'semi': ['non-structural'], 
            'full': ['non-structural', 'structural'], 
            'extreme': ['non-structural', 'structural', 'foundation']
        }
        strucTypes_all = ['non-structural', 'structural', 'foundation']
        strucTypes_demSites = strucTypes_forCircParam_dict[self.model.circularity_type] 
        strucTypes_suppliers = [i for i in strucTypes_all if i not in strucTypes_demSites]
        if self.model.modularity_type == 'full': 
            strucTypes_suppliers = strucTypes_suppliers + ['non-structural']
            # if 'structural' in strucTypes_suppliers: 
            #     strucTypes_suppliers.remove('structural')
            # if 'structural' in strucTypes_demSites:
            #     strucTypes_demSites.remove('structural')
        return strucTypes_suppliers, strucTypes_demSites
    
    def _make_matRequest_triaged(self, strucTypes): 
        '''make separate materials_requests for demSites and suppliers'''
        matRequest_triaged = {}
        for strucType in strucTypes:
            matRequest_triaged[strucType] = {}
            mat_amounts = self.materials_request[strucType]
            for mat, amount in mat_amounts.items(): 
                if mat == 'modules': 
                    continue
                matRequest_triaged[strucType][mat] = matRequest_triaged[strucType].get(mat, 0) + amount
        matRequest_triaged = {k: v for k, v in matRequest_triaged.items() if v!= 0}
        return matRequest_triaged
        
    def simplify_matRequest_forDemolitionSites(self): 
        '''convert mat names in materials_request_forDemSites to match mat names in demolition_sites_df'''
        
        materials_request_old = self.materials_request_forDemSites
        materials_request_new = {}
        
        for strucType, mat_amounts in materials_request_old.items(): 
            for mat, amount in mat_amounts.items(): 
                materials_request_new[mat] = materials_request_new.get(mat, 0) + amount
        
        self.materials_request_forDemSites = materials_request_new
                               
    def collect_materials_fromDemolitionSites(self): 
        '''collect materials from randomly selected demolition sites'''
        
        materials_collected = {key: 0 for key in self.materials_request_forDemSites.keys()}
        demSites = self.model.demolition_sites_df
        demSites = demSites[demSites.nearestMacroHub_id == self.unique_id]
        mats_collected = []
        
        for matName_conSite, request_amount in self.materials_request_forDemSites.items(): 
            matName_demSite = self._get_matName_demSite(matName_conSite)
            self.distances_all = []
            self.nTrips_all = []
            self.emissions_all = []
            
            while True: 
                demSite = demSites.sample(1).iloc[0]
                collect_tons = self._calc_collect_tons(demSite, matName_demSite, matName_conSite, request_amount, materials_collected)
                vehicle, capacity = self._get_vehicle_forDemSite(demSite, matName_demSite)
                nTrips, emissions = self._record_emissions(demSite, matName_conSite, collect_tons, vehicle, capacity)
                if vehicle.transportation_network == 'road': 
                    self._record_road_impact(demSite, nTrips, vehicle, capacity)
                                                   
                # stop if enough materials have been collected 
                materials_collected[matName_conSite] += collect_tons
                if materials_collected[matName_conSite] >= request_amount: 
                    break
                
            # record mats collected for print statement below
            mats_collected.append(matName_conSite)
        # print(f'{self.hubType} hub {self.unique_id} has collected the following materials from demolition sites: \n{mats_collected}')
        
    def _get_matName_demSite(self, matName_conSite): 
        conversion_df = self.model.materialNames_conversion
        matName_demSite = conversion_df[conversion_df.name_from_conSiteData == matName_conSite].name_from_demSiteData.iloc[0]
        return matName_demSite
    
    def _calc_collect_tons(self, demSite, matName_demSite, matName_conSite, request_amount, materials_collected): 
        available_tons = demSite[matName_demSite] 
        still_needed = request_amount - materials_collected[matName_conSite]
        collect_tons = still_needed if available_tons >= still_needed else available_tons
        return collect_tons
    
    def _get_vehicle_forDemSite(self, demSite, matName_demSite): 
        '''make vehicle capacities for demolition sites by converting mat names in vehicles_info
        into mat names in demolition_sites_df'''
        
        # get info for vehicle based on model params  
        if self.model.network_type == 'water' and self.waterbound and demSite.waterbound: 
            transportation_network = 'water'
            vehicle_type = 'water'
        else: # road network is used 
            transportation_network = 'road'
            if self.model.truck_type == 'semi': 
                vehicle_type = 'electric' if self.inA10 or demSite.inA10 else 'diesel'
            else: 
                vehicle_type = self.model.truck_type 
        
        # select vehicle 
        vehicles_df = self.model.vehicles_info_demSites
        vehicle = vehicles_df[(vehicles_df.transportation_network == transportation_network) & 
                              (vehicles_df.vehicle_type == vehicle_type)].iloc[0]
        capacity = vehicle[f'capacity_{matName_demSite}'] 
        
        return vehicle, capacity # need this for self.collect_materials_fromDemolitionSites() 
    
    def _record_emissions(self, demSite, matName_conSite, collect_tons, vehicle, capacity): 
        distance = demSite.nearestMacroHub_dist
        nTrips = math.ceil(collect_tons / capacity)
        emissions = vehicle.emissions_perKm_tCO2 * distance * nTrips * 2
        
        self.distances_all.append(distance)
        self.nTrips_all.append(nTrips*2)
        self.emissions_all.append(emissions)

        self.model.emissions_d2h += emissions
        self.model.emissions_per_material[matName_conSite] = self.model.emissions_per_material.get(matName_conSite, 0) + emissions
        self.demolition_site_ids.append(demSite.unique_id)
        
        self.model.emissions_NOX += vehicle.emissions_perKm_gNOX * distance * nTrips * 2
        self.model.emissions_PM2p5 += vehicle.emissions_perKm_gPM2p5 * distance * nTrips * 2
        self.model.emissions_PM10 += vehicle.emissions_perKm_gPM10 * distance * nTrips * 2
        
        return nTrips, emissions
        
    def _record_road_impact(self, demSite, nTrips, vehicle, capacity): 
        # record roads used
        roadMatrix = self.model.road_matrix_d2h
        road_ids = roadMatrix[(roadMatrix[:, 1] == self.unique_id) & (roadMatrix[:, 0] == demSite.unique_id)][0][2]
        mask = self.model.roads_used['osmid'].isin(road_ids)
        self.model.roads_used.loc[mask, 'nTrips'] += nTrips

        # record road damage
        nAxels = vehicle.nAxels
        weight = capacity / nAxels 
        damage = (weight ** 4) * nTrips 
        self.model.roads_used.loc[mask, 'damage'] += damage
    
    def find_suppliers(self): 
        '''this function is only run by macro hubs - see Hub.step()
        select supplier agent and distance based on location type (national / international)
        self.materials_request_forSuppliers = {'mat': 123, 'mat': 456 ... }
        self.suppliers = {'timber': {'agent': agentObject, 'distance': 14312}, ... etc}'''
               
        self.suppliers = {}
        mat_info = self.model.materials_logistics_info
        
        matList = []
        for strucType, mat_amounts in self.materials_request_forSuppliers.items(): 
            for mat in mat_amounts.keys(): 
                matList.append(mat)
        matList = list(set(matList))
        
        for mat in matList:
            self.suppliers[mat] = {}
            supplier = [s for s in self.model.suppliers if s.material_type == mat][0]
            self.suppliers[mat]['agent'] = supplier 
            self.suppliers[mat]['distance'] = haversine(self.coords, supplier.coords) # supplier.distance_fromAms
                        
    def collect_materials_fromSupplier(self): 
        '''this function is only run by macro hubs - see Hub.step()
        collect materials from factory supplier (national / international)'''
        mats_collected = []
        # print(f'request for suppliers: \n{self.materials_request_forSuppliers}')
        for strucType, mat_amounts in self.materials_request_forSuppliers.items():
            for mat, amount in mat_amounts.items(): 
                if amount == 0: 
                    continue
                mats_collected.append(f'{strucType} {mat}')
                # get info for calculating emissions and road usage 
                supplier = self.suppliers[mat]['agent']
                distance = self.suppliers[mat]['distance']
                if self.model.network_type == 'water' and not self.waterbound: 
                    network_type = 'road'
                else: 
                    network_type = self.model.network_type
                vehicle = self._select_vehicle(mat, network_type, 'diesel')
                capacity = vehicle['capacity_modules_2D'] if mat == 'modules' else vehicle[f'capacity_{mat}'] # * 0.8
                nTrips = math.ceil(amount / capacity)

                # record road usage road network is used  
                if self.model.network_type == 'road' or (self.model.network_type == 'water' and not self.waterbound): 
                    roadMatrix = self.model.road_matrix_s2h
                    road_ids = roadMatrix[(roadMatrix[:, 0] == supplier.unique_id) & (roadMatrix[:, 1] == self.unique_id)][0][2]
                    mask = self.model.roads_used['osmid'].isin(road_ids)
                    self.model.roads_used.loc[mask, 'nTrips'] += nTrips

                    # record road damage
                    nAxels = vehicle['nAxels']
                    weight = capacity / nAxels 
                    damage = (weight ** 4) * nTrips 
                    self.model.roads_used.loc[mask, 'damage'] += damage

                # record emissions, materials received, and suppliers used 
                emissions = vehicle['emissions_perKm_tCO2'] * distance * nTrips * 2
                self.model.emissions_s2h += emissions
                self.model.emissions_per_material[mat] = self.model.emissions_per_material.get(mat, 0) + emissions
                self.materials_received[mat] += amount
                self.supplier_ids.append(supplier.unique_id)
                
                self.model.emissions_NOX += vehicle.emissions_perKm_gNOX * distance * nTrips * 2
                self.model.emissions_PM2p5 += vehicle.emissions_perKm_gPM2p5 * distance * nTrips * 2
                self.model.emissions_PM10 += vehicle.emissions_perKm_gPM10 * distance * nTrips * 2

        
    def send_materials_toClient(self): 
        '''send materials to client (either construction sites or micro hubs) 
        self.materials_toSend = {site_id: {'foundation': {'timber': 123, ... }, ... }, ... }'''
        mats_sent = []
        # for each client (either construction site or micro hub): 
        for client_id, mat_toSend_dict in self.materials_toSend.items(): 
            
            # get client and vehicle info
            client = self.clients[client_id]['agent']
            distance = self.clients[client_id]['distance']
            
            # get vehicle based on params 
            fuel_type = self.model.truck_type
            network_type = self.model.network_type
            if self.model.network_type == 'water' and self.waterbound and client.waterbound: 
                network_type = 'water'
            else: # network_type == 'road' or 'rail', or not waterbound: 
                network_type = 'road'
                if self.model.truck_type == 'semi': 
                    fuel_type = 'electric' if client.inA10 or self.inA10 else 'diesel'
                else: # truck_type == 'none' or 'full'
                    fuel_type = self.model.truck_type      
            
            # determine path used
            roadMatrix = self.model.road_matrix_h2hc
            road_ids = roadMatrix[(roadMatrix[:, 0] == self.unique_id) & (roadMatrix[:, 1] == client.unique_id)][0][2]
            road_ids_str = [','.join(map(str, r)) if isinstance(r, list) else str(r) for r in road_ids]
            # mask = self.model.roads_used['osmid'].isin(road_ids_str)
            mask = self.model.roads_used['osmid'].apply(lambda x: x in road_ids_str)

            damages = []
            n_trips_list = []
            for strucType, mat_amounts in mat_toSend_dict.items(): 
                for mat, amount in mat_amounts.items(): 
                    mats_sent.append(f'{strucType} {mat}')
                    vehicle = self._select_vehicle(mat, network_type, fuel_type)

                    # record emissions 
                    capacity = vehicle['capacity_modules_3D'] if mat == 'modules' else vehicle[f'capacity_{mat}']
                    nTrips = math.ceil(amount / capacity)
                    emissions = vehicle.emissions_perKm_tCO2 * distance * nTrips * 2
                    self.model.emissions_h2c += emissions
                    self.model.emissions_per_material[mat] = self.model.emissions_per_material.get(mat, 0) + emissions
                    
                    self.model.emissions_NOX += vehicle.emissions_perKm_gNOX * distance * nTrips * 2
                    self.model.emissions_PM2p5 += vehicle.emissions_perKm_gPM2p5 * distance * nTrips * 2
                    self.model.emissions_PM10 += vehicle.emissions_perKm_gPM10 * distance * nTrips * 2
                    
                    if network_type == 'road': 
                        # record roads used, road damage
                        nAxels = vehicle.nAxels
                        weight = capacity / nAxels 
                        damage = (weight ** 4) * nTrips 
                        if self.model.network_type == 'water': 
                            if self.waterbound and client.waterbound: 
                                damage = 0
                        damages.append(damage)
                        n_trips_list.append(nTrips)
                    
                    # record  materials received, client ids 
                    if type(client) is ConstructionSite: 
                        client.materials_received[strucType].setdefault(mat, 0)
                        client.materials_received[strucType][mat] += amount
                    else: # if client == micro hub: 
                        client.materials_received.setdefault(mat, 0)
                        client.materials_received[mat] += amount
                    self.client_ids.append(client_id)
                    
            self.model.roads_used.loc[mask, 'damage'] += sum(damages)
            self.model.roads_used.loc[mask, 'nTrips'] += sum(n_trips_list)
        # print(f'{self.hubType} hub {self.unique_id} sent the following materials to its clients: {set(mats_sent)}')

    def _select_vehicle(self, mat, network_type, fuel_type):
        logistics_info = self.model.materials_logistics_info
        mat = 'modules_2D' if mat == 'modules' else mat # doesn't matter if it's modules_2D or 3D, they use the same truck. And we're only selecting a truck here.
        logistics_info = logistics_info[logistics_info.material == mat].iloc[0]
        vehicle_info = self.model.vehicles_info

        if network_type == 'water': 
            fuel_type = 'water'
            if logistics_info.water_usage == 'no':
                network_type = 'road'
                fuel_type = 'diesel'
        elif network_type == 'rail': 
            fuel_type = 'rail'
            if logistics_info.rail_usage == 'no':
                network_type = 'road'
                fuel_type = 'diesel'

        vehicle = vehicle_info[
            ((vehicle_info.transportation_network != 'road') | 
             (vehicle_info.vehicle_name == logistics_info.vehicle_name)) & 
            (vehicle_info.transportation_network == network_type) & 
            (vehicle_info.vehicle_type == fuel_type)
        ].iloc[0]

        return vehicle
    
class Supplier(Agent): 
    # supplier = Supplier(row.unique_id, self, row.material, coords)
    def __init__(self, unique_id, model, material, coords): 
        super().__init__(unique_id, model)
        self.material = material
        self.location_type = material
        self.coords = coords
        self.clients = {}
        self.waterbound = 0
        self.material_type = material
        
    def step(self): 
        self.find_clients() 
        self.calc_materials_toSend() 
        if self.materials_toSend: 
            # print(f'{self.material_type} supplier is sending materials to site ... ')
            self.send_materials_supplierToSite() 
    
    def find_clients(self): 
        '''self.clients = list of client agents [agent, agent, agent ...]
        clients are only construction sites, not hubs, because logistics
        between suppliers and hubs are handled by hubs, not suppliers.'''
        sites = [site for site in self.model.construction_sites if site.materials_request_forSuppliers]
        clients = []
        for site in sites: 
            if self.model.hub_network != 'none': 
                matReq = site.materials_request_forSuppliers
            else: 
                matReq = site.materials_request_all
            for mat_amounts in matReq.values(): 
                for mat in mat_amounts.keys(): 
                    if mat == self.material_type: 
                        clients.append(site)
        self.clients = clients
                        
    def calc_materials_toSend(self): 
        '''self.materials_toSend = {id: 123 ... }'''
        self.materials_toSend = {}
        for client in self.clients: 
            if self.model.hub_network != 'none': 
                mat_req = client.materials_request_forSuppliers
            else: 
                mat_req = client.materials_request_all
            self.materials_toSend[client.unique_id] = {}
            for strucType, mat_amounts in mat_req.items(): 
                self.materials_toSend[client.unique_id][strucType] = {} 
                for mat, amount in mat_amounts.items(): 
                    if mat == self.material_type: 
                        self.materials_toSend[client.unique_id][strucType][mat] = self.materials_toSend[client.unique_id][strucType].get(mat, 0) + amount 
        
    def send_materials_supplierToSite(self): 
        client_dict = {c.unique_id: c for c in self.clients}
        
        for client_id, mat_toSend_dict in self.materials_toSend.items(): 
            client = client_dict[client_id]
            distance = haversine(client.coords, self.coords)
            mask = self._get_mask_forRoadsUsed(self.unique_id, client_id)

            nTrips_total = []
            damage_total = []
            for strucType, mat_amounts in mat_toSend_dict.items(): 
                for mat, amount in mat_amounts.items():  
                    vehicle = self._select_vehicle(mat, self.model.network_type, self.model.truck_type)
                    capacity = vehicle[f'capacity_modules_3D'] if mat == 'modules' else vehicle[f'capacity_{mat}'] * 0.3 
                    nTrips = math.ceil(amount / capacity)
                    client.materials_received[strucType][mat] += amount
                    self._record_emissions(vehicle, mat, amount, distance, nTrips)
                    self._record_road_impact(nTrips_total, damage_total, mat, amount, vehicle)
                    
            self.model.roads_used.loc[mask, 'nTrips'] += sum(nTrips_total)
            self.model.roads_used.loc[mask, 'damage'] += sum(damage_total)

    def _select_vehicle(self, mat, network_type, fuel_type):
        logistics_info = self.model.materials_logistics_info
        logistics_info = logistics_info[logistics_info.material == mat].iloc[0]
        vehicle_info = self.model.vehicles_info

        fuel_type = 'diesel'
        if network_type == 'water': 
            fuel_type = 'water'
            if logistics_info.water_usage == 'no':
                network_type = 'road'
                fuel_type = 'diesel'
        elif network_type == 'rail': 
            fuel_type = 'rail'
            if logistics_info.rail_usage == 'no':
                network_type = 'road'
                fuel_type = 'diesel'

        vehicle = vehicle_info[
            ((vehicle_info.transportation_network != 'road') | 
             (vehicle_info.vehicle_name == logistics_info.vehicle_name)) & 
            (vehicle_info.transportation_network == network_type) & 
            (vehicle_info.vehicle_type == fuel_type)
        ].iloc[0]

        return vehicle
    
    def _get_mask_forRoadsUsed(self, self_id, client_id): 
        roadMatrix = self.model.road_matrix_s2c
        road_ids = roadMatrix[(roadMatrix[:, 0] == self_id) & 
                              (roadMatrix[:, 1] == client_id)][0][2]
        mask = self.model.roads_used['osmid'].isin(road_ids)
        return mask 
    
    def _record_emissions(self, vehicle, mat, amount, distance, nTrips): 
        emissions = vehicle.emissions_perKm_tCO2 * distance * nTrips * 2
        self.model.emissions_s2c += emissions
        self.model.emissions_per_material[mat] = self.model.emissions_per_material.get(mat, 0) + emissions
        
        self.model.emissions_NOX += vehicle.emissions_perKm_gNOX * distance * nTrips * 2
        self.model.emissions_PM2p5 += vehicle.emissions_perKm_gPM2p5 * distance * nTrips * 2
        self.model.emissions_PM10 += vehicle.emissions_perKm_gPM10 * distance * nTrips * 2
        
    def _record_road_impact(self, nTrips_total, damage_total, mat, amount, vehicle): 
        capacity = vehicle[f'capacity_modules_3D'] if mat == 'modules' else vehicle[f'capacity_{mat}'] * 0.3 

        nTrips = math.ceil(amount / capacity)
        nTrips_total.append(nTrips)

        nAxels = vehicle.nAxels
        weight = capacity / nAxels 
        damage = (weight ** 4) * nTrips 
        damage_total.append(damage)

In [100]:
from mesa import Model
from mesa.datacollection import DataCollector
class Model(Model):
    def __init__(self, parameters_dict): 
        '''create construction sites, hubs, and vehicles'''
        super().__init__()
        self.schedule = BaseScheduler(self)
        self.emissions_s2h = 0
        self.emissions_h2c = 0 
        self.emissions_s2c = 0
        self.emissions_d2h = 0
        self.emissions_NOX = 0
        self.emissions_PM2p5 = 0
        self.emissions_PM10 = 0
        self.emissions_per_material = {}
        self.emissions_glass = {}
        self.roads_used = gpd.read_file('data/data_cleaned/ams_roads_edges.shp')
        self.roads_used['damage'] = 0 
        self.roads_used['damage'] = self.roads_used['damage'].astype(float)
        self.datacollector = DataCollector(
            model_reporters = {
                'emissions_s2h': lambda m: m.emissions_s2h, 
                'emissions_h2c': lambda m: m.emissions_h2c, 
                'emissions_s2c': lambda m: m.emissions_s2c, 
                'emissions_d2h': lambda m: m.emissions_d2h, 
                'emissions_total': lambda m: m.emissions_s2h + m.emissions_h2c + m.emissions_s2c + m.emissions_d2h,
                'emissions_per_material': lambda m: dict(m.emissions_per_material), 
                'emissions_glass': lambda m: dict(m.emissions_glass),
                'emissions_gNOX': lambda m: m.emissions_NOX, 
                'emissions_gPM2p5': lambda m: m.emissions_PM2p5, 
                'emissions_gPM10': lambda m: m.emissions_PM10
            }
        )
        
        self.load_data()
        self.load_material_info()
        self.add_parameters(parameters_dict) 
        
        '''
        DO NOT CHANGE ORDER OF AGENT CREATION BELOW. 
        od matrices (created externally in dataPrep.ipynb to save time) depends on 
        agent.unique_ids, which change if the order of agent creation is changed. 
        '''
        self.create_constructionSites()
        self.create_suppliers() 
        self.create_hubs()
        
        self.create_od_matrix_h2c()
        self.create_od_matrix_h2h()
        self.assign_hubs_to_sites()
        self.assign_hubs_to_hubs()
        
        if self.circularity_type != 'none': 
            self.create_od_matrix_d2h()
            self.assign_hubs_to_demolition_sites()
                    
    def load_data(self): 
        self.construction_sites_df = gpd.read_file('data/data_cleaned/construction_sites.shp')
        self.hubs_df = gpd.read_file('data/data_cleaned/hubs.shp')
        self.suppliers_df = gpd.read_file('data/data_cleaned/suppliers.shp')
        self.demolition_sites_df = gpd.read_file('data/data_cleaned/demolition_sites.shp')
        self.vehicles_info = pd.read_csv('data/data_cleaned/vehicles_info.csv')
        self.vehicles_info_demSites = pd.read_csv('data/data_cleaned/vehicles_info_demSites.csv')
        self.build_info = pd.read_csv('data/data_cleaned/buildingType_info.csv')
        self.materials_logistics_info = pd.read_csv('data/data_cleaned/materials_logistics_info.csv')
        self.materialNames_conversion = pd.read_csv('data/data_cleaned/materialNames_conversion.csv')
        self.materials_list = list(self.build_info.material.unique())
        self.road_matrix_h2hc = np.load('data/data_cleaned/roadOsmIds_matrix_h2hc.npy', allow_pickle=True)
        self.road_matrix_d2h = np.load('data/data_cleaned/roadOsmIds_matrix_d2h.npy', allow_pickle=True)
        self.road_matrix_s2h = np.load('data/data_cleaned/roadOsmIds_matrix_s2h.npy', allow_pickle=True)
        self.road_matrix_s2c = np.load('data/data_cleaned/roadOsmIds_matrix_s2c.npy', allow_pickle=True)

        self.construction_sites = []
        self.hubs = []
        self.suppliers = []
        self.trucks_urban = []
        self.vehicles_international = []
        
    def load_material_info(self): 
        df = self.materials_logistics_info
        self.materials_forHubs = df[df.hub_usage == 'yes'].material.to_list()
        self.materials_forHubs = self.materials_forHubs + ['modules']
        self.materials_forSuppliers = df[df.hub_usage == 'no'].material.to_list()
        
    def add_parameters(self, parameters_dict): 
        self.network_type = parameters_dict['network_type']
        self.truck_type = parameters_dict['truck_type']
        self.biobased_type = parameters_dict['biobased_type']
        self.modularity_type = parameters_dict['modularity_type']
        self.hub_network = parameters_dict['hub_network']
        self.circularity_type = parameters_dict['circularity_type']
        self.model_boundary = parameters_dict['model_boundary']
        self.parameters_dict = parameters_dict
    
    def create_constructionSites(self): 
        for i, row in self.construction_sites_df.iterrows(): 
            coords = (row.geometry.y, row.geometry.x)
            site = ConstructionSite(row.unique_id, self, row.buildType, 
                                    coords, row.inA10, row.waterbound, 
                                    row.start_year, row.end_year, row.multiplier)
            self.schedule.add(site)
            self.construction_sites.append(site)
            
    def create_suppliers(self): 
        for i, row in self.suppliers_df.iterrows(): 
            coords = (row.geometry.y, row.geometry.x)
            supplier = Supplier(row.unique_id, self, row.material, coords)
            self.schedule.add(supplier)
            self.suppliers.append(supplier)
    
    def create_hubs(self): 
        if self.hub_network == 'centralized': 
            hub_type = ['macro', 'macro_assembly']
        elif self.hub_network == 'decentralized': 
            hub_type = ['micro', 'macro', 'macro_assembly']
        elif self.hub_network == 'none': 
            hub_type = ['macro']
        for i, row in self.hubs_df.iterrows(): 
            if row.hub_type in hub_type: 
                coords = (row.geometry.y, row.geometry.x)
                hub = Hub(row.unique_id, self, row.hub_type, coords, row.inA10, row.waterbound)
                self.schedule.add(hub)
                self.hubs.append(hub)
                
    def get_capacity(self, network_type, truck_type): 
        '''capacity_dict = {'timber': 20, 'concrete': 25, ... }
        emissions_perTonKm = 0.0009 (emissions per km for a particular vehicle)'''
        v = self.vehicles_info.copy()
        v = v[(v.transportation_network == network_type) & (v.vehicle_type == truck_type)]
        capacity_dict = {}
        for mat in self.materials_list + ['modules']: 
            capacity = v[f'capacity_{mat}'].iloc[0]
            capacity_dict[mat] = capacity
        emissions_perTonKm = v.emissions_perTonKm.iloc[0]
        nAxels = v.nAxels.iloc[0]
        return capacity_dict, emissions_perTonKm, nAxels
    
    # this needs to be changed to real distance od matrix 
    # add this in data prep 
    def create_od_matrix_h2c(self): 
        '''This od matrix was made in dataPrep.ipynb. The ids correspond to the 
        agent unique ids in the agent based model. If the input data for construction sites
        and hubs changes, this od matrix needs to change accordingly in dataPrep.ipynb.'''
        self.od_matrix_h2c = np.load('data/data_cleaned/od_matrix_h2c.npy')
    
    def create_od_matrix_h2h(self): 
        '''This od matrix was made in dataPrep.ipynb. The ids correspond to the 
        agent unique ids in the agent based model. If the input data for construction sites
        and hubs changes, this od matrix needs to change accordingly in dataPrep.ipynb.'''
        self.od_matrix_h2h = np.load('data/data_cleaned/od_matrix_h2h.npy')
        
    def create_od_matrix_d2h(self): 
        '''This od matrix was made in dataPrep.ipynb. The ids correspond to the 
        agent unique ids in the agent based model. If the input data for construction sites
        and hubs changes, this od matrix needs to change accordingly in dataPrep.ipynb.'''
        self.od_matrix_d2h = np.load('data/data_cleaned/od_matrix_d2h.npy')
                
    def assign_hubs_to_sites(self):
        od = self.od_matrix_h2c
        nMacroHubs = len([hub for hub in self.hubs if hub.hubType == 'macro'])
        for site in self.construction_sites: 
            site_od = od[od[:, 1] == site.unique_id]
            if self.hub_network == 'decentralized': 
                site.nearestHub_id = int(site_od[np.argmin(site_od[:, 2]), 0])
                site.nearestHub_dist = site_od[np.argmin(site_od[:, 2]), 2]
            site_od_macro = site_od[:nMacroHubs]
            site.nearestMacroHub_id = int(site_od_macro[np.argmin(site_od_macro[:, 2]), 0])
            site.nearestMacroHub_dist = site_od_macro[np.argmin(site_od_macro[:, 2]), 2]
                        
    def assign_hubs_to_hubs(self): 
        od = self.od_matrix_h2h
        for hub in self.hubs: 
            hub_od = od[od[:, 1] == hub.unique_id]
            hub.nearestMacroHub_id = int(hub_od[np.argmin(hub_od[:, 2])][0])
            hub.nearestMacroHub_dist = hub_od[np.argmin(hub_od[:, 2])][2]
            
    def assign_hubs_to_demolition_sites(self): 
        od = self.od_matrix_d2h
        def func(row): 
            unique_id = row.unique_id
            site_od = od[od[:, 0] == unique_id]
            # nearest hub out of all hubs
            row['nearestHub_id'] = int(site_od[np.argmin(site_od[:, 2]), 1])
            row['nearestHub_dist'] = site_od[np.argmin(site_od[:, 2]), 2]
            # nearest macroHub
            macroHubIds = [h.unique_id for h in self.hubs if h.hubType == 'macro']
            site_od_macro = site_od[np.isin(site_od[:, 1], macroHubIds)]
            row['nearestMacroHub_id'] = int(site_od_macro[np.argmin(site_od_macro[:, 2]), 1])
            row['nearestMacroHub_dist'] = site_od_macro[np.argmin(site_od_macro[:, 2]), 2]
            return row
        self.demolition_sites_df = self.demolition_sites_df.apply(lambda row: func(row), axis=1)
            
    def step(self): # each model.step() is a year
        self.year = self.schedule.steps + 2023
        self.schedule.step()
        self.calc_emissions()
        self.datacollector.collect(self)
    
    def calc_emissions(self): 
        self.emissions = round(self.emissions_h2c + self.emissions_s2h + self.emissions_s2c + self.emissions_d2h)
    
    def visualize(self): 
        emissions_text = self.display_total_emissions()
        results_df = self.make_results_df()
        fig_emissions = self.display_emissions_chart()
        fig_emissions_perMaterial = self.display_emissions_chart_byMaterial()
        fig_materials_all = self.display_materials_chart(hub=False)
        fig_materials_hubs = self.display_materials_chart(hub=True)
        map_html = self.display_folium_html()

        return emissions_text, fig_emissions, fig_emissions_perMaterial, fig_materials_all, fig_materials_hubs, map_html, results_df
    
    def make_results_forCaspar(self): 
        results_df = self.make_results_df()
        shps_all = self.display_folium_html(getshps=True) # construction, demolition, hubs, suppliers, roadsUsed
        emissions_df = self.display_emissions_chart(caspar=True)
        emissions_perMaterial_df = self.display_emissions_chart_byMaterial(caspar=True)
        tons_byMaterialType = self.display_materials_chart(caspar=True)[0]
        tons_byCircularityType = self.display_materials_chart(caspar=True)[1]

        return results_df, shps_all, emissions_df, emissions_perMaterial_df, tons_byMaterialType, tons_byCircularityType
        
    def make_results_df(self): 
        total_emissions = round(self.emissions_h2c + self.emissions_s2h + self.emissions_s2c + self.emissions_d2h)
        roads_used = self.roads_used
        total_distance_km_mra = self._calc_total_distance_km(roads_used)
        results_dict = {
            'result_name': ['co2', 'NOX', 'PM2.5', 'PM10', 'logistic movements'], 
            'value': [total_emissions, self.emissions_NOX, self.emissions_PM2p5, self.emissions_PM10, total_distance_km_mra], 
            'unit': ['tons', 'g', 'g', 'g', 'km']
        }
        results_df_model = pd.DataFrame(results_dict)
        results_df_model['area'] = 'whole model'
        
        # make results for MRA, ZO 
        total_distance_km_mra = self._calc_total_distance_km(roads_used)
        roads_used_zo = self._get_roads_used(zuidOost=True)
        total_distance_km_zo = self._calc_total_distance_km(roads_used_zo)

        results_df_mra = self._calc_emissions_from_totalDist(total_distance_km_mra, 'MRA')
        results_df_zo = self._calc_emissions_from_totalDist(total_distance_km_zo, 'zuid-oost')
        
        results_df = pd.concat([results_df_model, results_df_mra, results_df_zo])
        
        return results_df
    
    def _calc_emissions_from_totalDist(self, total_distance_km, area_name): 
        emissions_co2 = total_distance_km * 0.001539 # tCO2 emissions per km for trucks
        emissions_NOX = total_distance_km * 5.71 # gNOX per km
        emissions_pm2p5 = total_distance_km * 0.01805 # gPM2.5 per km 
        emissions_pm10 = total_distance_km * 0.0361 # gPM10 per km
        results_dict = {
            'result_name': ['co2', 'NOX', 'PM2.5', 'PM10', 'logistic movements'], 
            'value': [emissions_co2, emissions_NOX, emissions_pm2p5, emissions_pm10, total_distance_km], 
            'unit': ['tons', 'g', 'g', 'g', 'km']
        }
        results_df = pd.DataFrame(results_dict)
        results_df['area'] = area_name
        return results_df
    
    def display_total_emissions(self, caspar=False): 
        total_emissions = round(self.emissions_h2c + self.emissions_s2h + self.emissions_s2c + self.emissions_d2h)
        roads_used = self.roads_used
        roads_used_zo = self._get_roads_used(zuidOost=True)
        total_distance_km_mra = self._calc_total_distance_km(roads_used)
        million = '' if total_distance_km_mra<1000000 else 'million '
        total_distance_km_mra = total_distance_km_mra if total_distance_km_mra<1000000 else round(total_distance_km_mra/1000000,3)
        total_distance_km_zo = self._calc_total_distance_km(roads_used_zo)
        zuid_oost_text = self._make_zuidOost_text(total_distance_km_zo)
        emissions_text = (
            f'''
            CO2 emissions (suppliers to construction sites): {round(self.emissions_s2c)} tCO2
            CO2 emissions (suppliers to hubs): {round(self.emissions_s2h)} tCO2
            CO2 emissions (demSites to hubs): {round(self.emissions_d2h)} tCO2
            CO2 emissions (hubs to construction sites): {round(self.emissions_h2c)} tCO2
            CO2 emissions (total): {total_emissions} tCO2
            
            NOX emissions (total): {round(self.emissions_NOX/1000000)} tNOX
            PM2.5 emissions (total): {round(self.emissions_PM2p5/1000)} kg PM2.5
            PM10 emissions (total): {round(self.emissions_PM10/1000)} kg PM10
            
            total logistics movements within MRA: {total_distance_km_mra} {million}km
            
            {zuid_oost_text}
            '''
        )

        return emissions_text
    
    def _get_roads_used(self, zuidOost): 
        roads_used = self.roads_used 
        if zuidOost: 
            zuidOost = gpd.read_file('data/data_cleaned/zuidOost_buffer_500m.shp')
            roads_used_zuidOost = gpd.sjoin(roads_used, zuidOost, predicate='intersects')
            self.roads_used_zuidOost = roads_used_zuidOost
            roads_used = roads_used_zuidOost
        return roads_used
    
    def _calc_total_distance_km(self, roads_used):         
        roads_used = roads_used.to_crs('EPSG:28992') 
        roads_used['distance_m'] = roads_used.geometry.length
        roads_used['total_distance_m'] = roads_used.distance_m * roads_used.nTrips 
        total_distance_m = roads_used.total_distance_m.sum()
        total_distance_km = round(total_distance_m / 1000)
        return total_distance_km
    
    def _make_zuidOost_text(self, total_distance_km):
        million = '' if total_distance_km<1000000 else 'million '
        total_distance_km = total_distance_km if total_distance_km<1000000 else total_distance_km/1000000
        if self.model_boundary == 'zuid-oost': 
            emissions_co2 = total_distance_km * 0.001539 # tCO2 emissions per km for trucks
            emissions_NOX = total_distance_km * 5.71 # gNOX per km
            emissions_pm2p5 = total_distance_km * 0.01805 # gPM2.5 per km 
            emissions_pm10 = total_distance_km * 0.0361 # gPM10 per km
            text = f'''
            Zuid-Oost results: 
            total logistics movements: {round(total_distance_km)} {million}km
            CO2 emissions: {round(emissions_co2)} tCO2eq
            NOX emissions: {round(emissions_NOX/1000000, 3)} tNOX
            PM2.5 emissions: {round(emissions_pm2p5/1000, 3)} kg PM2.5
            PM10 emissions: {round(emissions_pm10/1000, 3)} kg PM10
            '''
        else: 
            text = ''
        return text
        
    def display_emissions_chart(self, caspar=False): 
        data = self.datacollector.get_model_vars_dataframe()
        data = data.reset_index(names='step')
        data.step = data.step.map(lambda x: x + 2023)
        data.rename(columns={'step': 'year'}, inplace=True)
        fig = px.line(data, x="year", 
                      y=['emissions_s2c', 'emissions_s2h', 'emissions_d2h', 'emissions_h2c', 'emissions_total'], 
                      title='emissions')
        fig.update_layout(height=500)  # or any desired height in pixels
        
        if caspar: 
            return data 
        else: 
            return fig
    
    def display_emissions_chart_byMaterial(self, caspar=False): 
        data = self.datacollector.get_model_vars_dataframe()
        df = data.copy()
        df = df[['emissions_per_material']]
        df = df.explode('emissions_per_material').reset_index(names='step')
        df = df.rename(columns={'emissions_per_material': 'material'})
        df = df.dropna()
        def get_emissions(row): 
            emissions_dict = data.iloc[row.step].emissions_per_material
            row['emissions'] = emissions_dict[row.material]
            return row
        df = df.apply(lambda row: get_emissions(row), axis=1)

        dfAdd = df[df.step == 1]
        dfAdd.step = 0
        dfAdd.emissions = 0
        df = pd.concat([dfAdd, df])
        df.step = df.step.map(lambda x: x + 2023)
        df.rename(columns={'step': 'year'}, inplace=True)

        fig = px.line(df, x='year', y='emissions', color='material', title='emissions per material')
        fig.update_layout(height=500)

        return df if caspar else fig
    
    def display_materials_chart(self, hub=False, caspar=False): 
        if hub: # display materials processed by hub(s)
            s = '' if self.model_boundary == 'zuid-oost' else 's'
            title_text = f'Materials processed by hub{s} in {self.model_boundary}'
            df_mat, total_tons = self._make_df_materials(hub=True)
            df_circ = self._make_df_circular(self.circularity_type, hub=True)
        else: # display all materials 
            title_text = 'Material composition for buildings in the MRA'
            df_mat, total_tons = self._make_df_materials(hub=False)
            df_circ = self._make_df_circular(self.circularity_type, hub=False)
        million = 'million ' if total_tons > 1000000 else ''
        total_tons = round(total_tons) if total_tons < 1000000 else round(total_tons / 1000000, 3)
        subtitle_text = f'total mass: {total_tons} {million}tons'
        
        fig_1 = px.pie(df_mat, values='tons', names='material', title='materials used')
        fig_2 = px.pie(df_circ, values='tons', names='circular', title='materials used')

        fig = make_subplots(rows=1, cols=2, 
                            subplot_titles=('by material type', 'by circularity type'),
                            specs=[[{'type':'domain'}, {'type':'domain'}]])
        fig.add_trace(fig_1.data[0], 1, 1)
        fig.add_trace(fig_2.data[0], 1, 2)
        fig.update_layout(title=f'{title_text}<br><sup>{subtitle_text}</sup>')
        for annotation in fig['layout']['annotations']: 
            annotation['font'] = dict(size=12)  # Set to desired font size

        return [df_mat, df_circ] if caspar else fig 
    
    def _make_df_materials(self, hub=False): 
        dfs = []
        materials_list = self.materials_forHubs if hub else self.materials_list + ['modules_2D', 'modules_3D'] 
        consite_agents_list = self.construction_sites if self.model_boundary != 'zuid-oost' else [c for c in self.construction_sites if c.nearestMacroHub_id == 374]
        for site in consite_agents_list: 
            df = site.material_composition_df
            df = df[df.material.isin(materials_list)]
            df = df.groupby(by='material').sum(numeric_only=True).reset_index()
            dfs.append(df)
        df_all = pd.concat(dfs).groupby('material').sum(numeric_only=True).reset_index() 
        self.df_mat = df_all
        total_tons = round(df_all.tons.sum())
        return self.df_mat, total_tons
    
    def _make_df_circular(self, circularity_type, hub=False): 
        circularity_dict = {
            'none': [], 
            'semi': ['non-structural'], 
            'full': ['non-structural', 'structural'], 
            'extreme': ['non-structural', 'structural', 'foundation']
        }

        dfs = []
        materials_list = self.materials_forHubs + self.materials_forSuppliers
        consite_agents_list = self.construction_sites
        if self.model_boundary == 'zuid-oost': 
            sites_zo = [c for c in consite_agents_list if c.nearestMacroHub_id == 374]
            consite_agents_list = sites_zo
            if hub: 
                materials_list = self.materials_forHubs

        for site in consite_agents_list: 
            df = site.material_composition_df
            df = df[df.material.isin(materials_list)]
            def decide_circularity(row): 
                bulk_materials = ['cement_cover-floor', 'concrete_in-situ', 'concrete_prefab', 'gravel_sand']
                row['circular'] = 'circular' if row.structural_type in circularity_dict[circularity_type] else 'not circular'
                if row.material in bulk_materials or row.material == 'modules': 
                    row['circular'] = 'not circular'
                return row 
            df = df.apply(lambda row: decide_circularity(row), axis=1)
            df = df.groupby('circular').sum(numeric_only=True).reset_index()
            
            dfs.append(df)
            
        self.df_circ = pd.concat(dfs).groupby('circular').sum(numeric_only=True).reset_index()
        return self.df_circ
        
    def display_folium_html(self, getshps=False): 

        shps_forCaspar = {}

        m = folium.Map([52.377231, 4.899288], zoom_start=11, tiles='cartodbpositron')
        
        # plot conditional shps 
        if self.network_type != 'road': 
            self._plotLines_nonRoad_networks(m, self.network_type)
            shps_forCaspar['nonRoad_networks'] = self._plotLines_nonRoad_networks(m, self.network_type, getshp=True)
        if self.hub_network != 'none':  
            if self.circularity_type != 'none': 
                self.plotPoints_demSites(m, '#bababa', 1)
                shps_forCaspar['demolition_sites'] = self.plotPoints_demSites(m, '#bababa', 1, getshp=True)
        if self.model_boundary == 'zuid-oost': 
            self._plot_zuidOost_boundary(m)
            shps_forCaspar['zuidOost_boundary'] = self._plot_zuidOost_boundary(m, getshp=True)
        
        # plot all shps
        self.plotLines_roadsUsed(m, self.roads_used)
        self.plotPoints(m, self.suppliers, 'grey', 1)
        self.plotPoints(m, self.construction_sites, '#e8aa00', 1)
        self.plotPoints(m, [h for h in self.hubs if h.hubType != 'micro' and h.clients], 'red', 8)
        self.plotPoints(m, [h for h in self.hubs if h.hubType == 'micro' and h.clients], 'red', 4)
        html_string = m._repr_html_()

        # add shps to shps_forCaspar
        shps_forCaspar['roadsUsed'] = self.plotLines_roadsUsed(m, self.roads_used, getshp=True)  
        shps_forCaspar['construction_sites'] = self.plotPoints(m, self.construction_sites, '#e8aa00', 1, getshp=True)
        shps_forCaspar['hubs_macro'] = self.plotPoints(m, [h for h in self.hubs if h.hubType != 'micro' and h.clients], 'red', 8, getshp=True)
        shps_forCaspar['hubs_micro'] = self.plotPoints(m, [h for h in self.hubs if h.hubType == 'micro' and h.clients], 'red', 4, getshp=True)
        shps_forCaspar['suppliers'] = self.plotPoints(m, self.suppliers, 'grey', 1, getshp=True)

        return shps_forCaspar if getshps else html_string
    
    def _plotLines_nonRoad_networks(self, m, network_type, getshp=False): 
        color = 'blue' if network_type == 'water' else 'brown'
        paths_plot = gpd.read_file(f'data/networkLines_{network_type}.gpkg') 
        
        def style_function(feature):
            return {'color': color,'weight': 2,'opacity': 1,'dashArray': '5, 5'}
        
        # print(f'adding {network_type} paths to map ...')
        folium.GeoJson(paths_plot, style_function=style_function).add_to(m)

        if getshp: 
            return paths_plot
                
    def plotPoints(self, m, agent_list, color, radius, getshp=False): 
        for agent in agent_list: 
            selected_color = color
            folium.CircleMarker(
                location=agent.coords, radius=radius, color=selected_color, fill_color=selected_color, 
                popup=folium.Popup(f'id: {agent.unique_id}', max_width=300, height=150), 
            ).add_to(m)

        if getshp: 
            df = gpd.GeoDataFrame()
            df['geometry'] = [Point(agent.coords[1], agent.coords[0]) for agent in agent_list]
            return df

    def plotLines_roadsUsed(self, m, df, getshp=False): 
        df = df[df.nTrips > 0]
        df = df.sort_values(by='nTrips')
        max_trips = df.nTrips.max()
        quantiles = df.nTrips.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).tolist()
        
        def get_color(value):
            if value <= quantiles[1]:  # corresponds to 0.2 quantile
                return 'rgb(255, 200, 200)'  # Light Pink
            elif value <= quantiles[2]:  # corresponds to 0.4 quantile
                return 'rgb(255, 150, 150)'  # Hot Pink
            elif value <= quantiles[3]:  # corresponds to 0.6 quantile
                return 'rgb(255, 100, 100)'     # Orange Red
            elif value <= quantiles[4]:  # corresponds to 0.8 quantile
                return 'rgb(220, 50, 50)'   # Crimson
            else:
                return 'rgb(255, 0, 0)'     # Bright Red
        def style_function(feature):
            nTrips = feature['properties']['nTrips']
            return {'weight': 2,'color': get_color(nTrips)}
        def popup_function(feature):
            nTrips = feature['properties']['nTrips']
            return folium.Popup(f'nTrips: {nTrips}')
        folium.GeoJson(
            df, style_function=style_function,
            tooltip=folium.GeoJsonTooltip(fields=['nTrips']),  # Add tooltip on hover
            popup=folium.GeoJsonPopup(fields=['nTrips'])      # Add popup on click
        ).add_to(m)

        import branca
        def rgb_to_hex(rgb):
            return '#{:02x}{:02x}{:02x}'.format(*rgb)
        colors_hex = [rgb_to_hex((255, 200, 200)), rgb_to_hex((255, 150, 150)), rgb_to_hex((255, 100, 100)), rgb_to_hex((220, 50, 50)), rgb_to_hex((255, 0, 0))]
        quantiles = quantiles[1:]
        colormap = branca.colormap.LinearColormap(colors=colors_hex, index=quantiles, vmin=quantiles[0], vmax=quantiles[-1])
        colormap.caption = 'Number of Trips'
        cmap_HTML = colormap._repr_html_()
        cmap_HTML = cmap_HTML.replace('<svg height="50" width="500">', '<svg id="cmap" height="50" width="500">', 1)
        # m.get_root().header.add_child(folium.Element(cmap_style))
        m.get_root().html.add_child(folium.Element(cmap_HTML))
        folium.map.LayerControl().add_to(m)

        if getshp: 
            return df
        
    def _calc_roadsUsed_zuidOost(self): 
        roads_used = self.roads_used 
        zuidOost = gpd.read_file('data/data_cleaned/zuidOost_buffer_500m.shp')
        self.roads_used_zuidOost = gpd.sjoin(roads_used, zuidOost, predicate='intersects')
               
    def _plot_zuidOost_boundary(self, m, getshp=False): 
        zuidOost = gpd.read_file('data/data_cleaned/zuidOost_buffer_500m.shp')
        folium.GeoJson(
            zuidOost, style_function=lambda feature: {
                'color': '#bababa', 'fillColor': '#bababa', 'fillOpacity': 0.2, 
                'dashArray': '5, 5', 'weight': 2
            }
        ).add_to(m)

        if getshp: 
            return zuidOost
                
    def plotLines_s2h(self, m): 
        if self.hub_network != 'none': 
            macroHubs = [hub for hub in self.hubs if hub.hubType == 'macro']
            for hub in self.hubs: 
                if hub.supplier_ids: 
                    for supplier_id in hub.supplier_ids: 
                        suppliers = self.suppliers if hub.hubType != 'micro' else macroHubs
                        supplier = [s for s in suppliers if s.unique_id == supplier_id][0]
                        folium.PolyLine(
                            locations=[supplier.coords, hub.coords], weight=1, 
                            color='#454545', dash_array='5'
                        ).add_to(m)
        else: 
            sites = self.construction_sites
            for supplier in self.suppliers: 
                for site in sites: 
                    folium.PolyLine(
                        locations=[supplier.coords, site.coords], weight=1, 
                        color='#454545', dash_array='5'
                    ).add_to(m)
                
    def plotLines_d2h(self, m):
        for hub in self.hubs: 
            coords_hub = hub.coords
            demSite_ids = list(set(hub.demolition_site_ids))
            for demSite_id in demSite_ids:
                demSites = self.demolition_sites_df
                demSite = demSites[demSites.unique_id == demSite_id].iloc[0]
                coord_demSite = (demSite.geometry.y, demSite.geometry.x)
                folium.PolyLine(
                    locations=[coords_hub, coord_demSite], weight=1, 
                    color='#454545', dash_array='5'
                ).add_to(m)
        
    def plotPoints_demSites(self, m, color, radius, getshp=False): 

        demSite_ids_all = []

        for hub in self.hubs: 
            demSite_ids = list(set(hub.demolition_site_ids))
            demSite_ids_all.extend(demSite_ids)
            for demSite_id in demSite_ids: 
                demSites = self.demolition_sites_df
                demSite = demSites[demSites.unique_id == demSite_id].iloc[0]
                coord = (demSite.geometry.y, demSite.geometry.x)
                folium.CircleMarker(
                    location=coord, radius=radius, color=color, 
                    popup=folium.Popup(f'id: {demSite.unique_id}', max_width=300)
                ).add_to(m)

        demSites_df = self.demolition_sites_df[self.demolition_sites_df.unique_id.isin(demSite_ids_all)]
        if getshp:
            return demSites_df
                
    def plotPoints_hubs(self, m, color):
        self.plotPoints(m, [hub for hub in self.hubs if hub.hubType == 'macro'], color, 5)
        if self.hub_network == 'decentralized': 
            for macroHub in [h for h in self.hubs if h.hubType == 'macro']: 
                for client_id in macroHub.client_ids: 
                    if client_id in [h.unique_id for h in self.hubs]: 
                        microHub = [h for h in self.hubs if h.unique_id == client_id][0]
                        folium.CircleMarker(
                            location=microHub.coords, radius=1, color=color, 
                            popup=folium.Popup(f'id: {microHub.unique_id}', max_width=300)
                        ).add_to(m)

In [4]:
import ipywidgets as widgets
from IPython.display import display

# Define your parameters and their options
params_options = {
    'hub_network': ['centralized', 'decentralized', 'none'],
    'network_type': ['road', 'water', 'rail'],
    'truck_type': ['diesel', 'semi', 'electric'],
    'biobased_type': ['biobased non-structural + structural elements', 
                      'conventional'],
    'modularity_type': ['conventional', 'non-structural modules'], 
    'circularity_type': ['circular non-structural elements', 
                      'circular non-structural + structural elements', 
                      'circular non-structural + structural + foundation elements', 
                      'conventional'],
    'model_boundary': ['metropolitan region of amsterdam', 'zuid-oost']
}

params_conversion = {
    'biobased_type': {'biobased non-structural + structural elements': 'full', 
                      'conventional': 'none'}, 
    'modularity_type': {'conventional': 'none', 
                        'non-structural modules': 'full'}, 
    'circularity_type': {'circular non-structural elements': 'semi', 
                      'circular non-structural + structural elements': 'full', 
                      'circular non-structural + structural + foundation elements': 'extreme', 
                      'conventional': 'none'}
}

def update_options_hub_network(change): 
    if change['new'] == 'none': 
        dropdowns['network_type'].options = ['road']
        dropdowns['truck_type'].options = ['diesel']
        dropdowns['circularity_type'].options = ['conventional']
        dropdowns['modularity_type'].options = ['conventional']
    else:
        # Reset to original options if 'none' is not selected
        dropdowns['network_type'].options = params_options['network_type']
        dropdowns['truck_type'].options = params_options['truck_type']
        dropdowns['circularity_type'].options = params_options['circularity_type']
        
def update_options_biobased_type(change): 
    if change['new'] == 'conventional': 
        dropdowns['modularity_type'].options = ['conventional']
    else:
        dropdowns['modularity_type'].options = params_options['modularity_type']

def update_options_modularity_type(change): 
    if change['new'] == 'non-structural modules': 
        dropdowns['circularity_type'].options = [ 
            'circular non-structural + structural elements', 
            'circular non-structural + structural + foundation elements', 
            'conventional'
        ]
    else:
        dropdowns['circularity_type'].options = params_options['circularity_type']

def update_options_circularity_type(change): 
    if change['new'] == 'circular non-structural elements': 
        dropdowns['modularity_type'].options = ['conventional']
    else:
        dropdowns['modularity_type'].options = params_options['modularity_type']

# Create dropdown widgets
dropdowns = {}
for key, options in params_options.items():
    layout = widgets.Layout(width='500px')
    style = {'description_width': '100px'}
    dropdowns[key] = widgets.Dropdown(options=options, description=key, 
                                      value=options[0], layout=layout, style=style)
    if key == 'hub_network': 
        dropdowns[key].observe(update_options_hub_network, names='value')
    if key == 'biobased_type': 
        dropdowns[key].observe(update_options_biobased_type, names='value')
    if key == 'modularity_type': 
        dropdowns[key].observe(update_options_modularity_type, names='value')
    display(dropdowns[key])

Dropdown(description='hub_network', layout=Layout(width='500px'), options=('centralized', 'decentralized', 'no…

Dropdown(description='network_type', layout=Layout(width='500px'), options=('road', 'water', 'rail'), style=De…

Dropdown(description='truck_type', layout=Layout(width='500px'), options=('diesel', 'semi', 'electric'), style…

Dropdown(description='biobased_type', layout=Layout(width='500px'), options=('biobased non-structural + struct…

Dropdown(description='modularity_type', layout=Layout(width='500px'), options=('conventional', 'non-structural…

Dropdown(description='circularity_type', layout=Layout(width='500px'), options=('circular non-structural eleme…

Dropdown(description='model_boundary', layout=Layout(width='500px'), options=('metropolitan region of amsterda…

In [5]:
# import ipywidgets as widgets
from IPython.display import display, HTML

button = widgets.Button(description="Run model!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output(wait=True)
        print('running model ... ')
        print('(be patient, this will take ~10-30 seconds)')
        
        parameters_dict = {}
        for name in dropdowns.keys(): 
            parameters_dict[name] = dropdowns[name].value
        for key, value in parameters_dict.items():
            if key in params_conversion:
                parameters_dict[key] = params_conversion[key][value]

        model = Model(parameters_dict)
        for i in range(2): 
            # each model.step() is a year. 
            # This model simulates 10 years: from 2023 to 2033. 
            model.step()

        emissions_text, fig_emissions, fig_emissions_perMaterial, fig_materials_all, fig_materials_hubs, map_html, results_df = model.visualize()
        
        def create_and_display_fig_widget(fig):
            fig_widget = widgets.Output(layout=widgets.Layout())
            with fig_widget:
                fig.show()
            return fig_widget
        fig_widget_emissions = create_and_display_fig_widget(fig_emissions)
        fig_widget_materials_all = create_and_display_fig_widget(fig_materials_all)
        fig_widget_materials_hubs = create_and_display_fig_widget(fig_materials_hubs)
        fig_widget_emissionsPerMaterial = create_and_display_fig_widget(fig_emissions_perMaterial)
             
        print(emissions_text)
        v_box = widgets.VBox([fig_widget_emissions, fig_widget_emissionsPerMaterial, fig_widget_materials_all, fig_widget_materials_hubs], layout=widgets.Layout(width='30%', height='600px'))
        map_widget = widgets.HTML(map_html, layout=widgets.Layout(width='70%'))
        h_box = widgets.HBox([v_box, map_widget])
        display(h_box)
        

button.on_click(on_button_clicked)

Button(description='Run model!', style=ButtonStyle())

Output()

# saving figures of Caspar

In [6]:
bimzec_scenarios = {
    1: {
        'hub_network': 'none',
        'network_type': 'road',
        'truck_type': 'diesel',
        'biobased_type': 'none',
        'modularity_type': 'none',
        'circularity_type': 'none',
        'model_boundary': 'metropolitan region of amsterdam'
    }, 
    2: {
        'hub_network': 'centralized',
        'network_type': 'water',
        'truck_type': 'diesel',
        'biobased_type': 'none',
        'modularity_type': 'none',
        'circularity_type': 'none',
        'model_boundary': 'metropolitan region of amsterdam'
    }, 
    3: {
        'hub_network': 'centralized',
        'network_type': 'water',
        'truck_type': 'semi',
        'biobased_type': 'none',
        'modularity_type': 'none',
        'circularity_type': 'none',
        'model_boundary': 'metropolitan region of amsterdam'
    }, 
    4: {
        'hub_network': 'centralized',
        'network_type': 'water',
        'truck_type': 'semi',
        'biobased_type': 'full',
        'modularity_type': 'none',
        'circularity_type': 'none',
        'model_boundary': 'metropolitan region of amsterdam'
    }, 
    5: {
        'hub_network': 'centralized',
        'network_type': 'water',
        'truck_type': 'electric',
        'biobased_type': 'full',
        'modularity_type': 'full',
        'circularity_type': 'none',
        'model_boundary': 'metropolitan region of amsterdam'
    }, 
    6: {
        'hub_network': 'centralized',
        'network_type': 'water',
        'truck_type': 'electric',
        'biobased_type': 'full',
        'modularity_type': 'full',
        'circularity_type': 'extreme',
        'model_boundary': 'metropolitan region of amsterdam'
    }
}

sample_scenario = {
    1: {
        'hub_network': 'none',
        'network_type': 'road',
        'truck_type': 'diesel',
        'biobased_type': 'none',
        'modularity_type': 'none',
        'circularity_type': 'none',
        'model_boundary': 'metropolitan region of amsterdam'
    },
}

In [18]:
for scenario_number, parameters_dict in bimzec_scenarios.items(): 
    print(f'\n\nscenario number: {scenario_number}')
    print(f'parameters: \n{parameters_dict}')

    print('running model ... ')
    model = Model(parameters_dict)
    for i in range(12): 
        print(f'model year: {i+2023}')
        model.step()

    print('visualizing results ... ')
    emissions_text, fig_emissions, fig_emissions_perMaterial, fig_materials_all, fig_materials_hubs, map_html, results_df = model.visualize()
    fig_dict = {
        'emissions': fig_emissions, 
        'emissions_perMaterial': fig_emissions_perMaterial, 
        'materialComposition_MRA': fig_materials_all, 
        'materialComposition_hubs': fig_materials_hubs, 
        'foliumMap': map_html, 
    }

    print('saving results ... ')
    # save plotly charts 
    import plotly.offline as pyo
    fig_name_list = list(fig_dict.keys())
    fig_name_list.remove('foliumMap')
    for fig_name in fig_name_list: 
        html_file_path = f'results/bimzecScenarios/s{scenario_number}_{fig_name}.html'
        pyo.plot(fig_dict[fig_name], filename=html_file_path, auto_open=False)

    # save folium map 
    with open(f'results/bimzecScenarios/s{scenario_number}_foliumMap.html', 'w', encoding='utf-8') as html_file: 
        html_file.write(fig_dict['foliumMap'])

    # save text file
    with open(f'results/bimzecScenarios/s{scenario_number}_resultsText.txt', 'w', encoding='utf-8') as text_file:
        text_file.write(emissions_text)
        
    # save csv of results
    results_df['scenario'] = scenario_number
    results_df.to_csv(f'results/bimzecScenarios/s{scenario_number}_results.csv', index=False)



scenario number: 1
parameters: 
{'hub_network': 'none', 'network_type': 'road', 'truck_type': 'diesel', 'biobased_type': 'none', 'modularity_type': 'none', 'circularity_type': 'none', 'model_boundary': 'metropolitan region of amsterdam'}
running model ... 
model year: 2023
model year: 2024
model year: 2025
model year: 2026
model year: 2027
model year: 2028
model year: 2029
model year: 2030
model year: 2031
model year: 2032
model year: 2033
model year: 2034
visualizing results ... 
saving results ... 


scenario number: 2
parameters: 
{'hub_network': 'centralized', 'network_type': 'water', 'truck_type': 'diesel', 'biobased_type': 'none', 'modularity_type': 'none', 'circularity_type': 'none', 'model_boundary': 'metropolitan region of amsterdam'}
running model ... 
model year: 2023
model year: 2024
model year: 2025
model year: 2026
model year: 2027
model year: 2028
model year: 2029
model year: 2030
model year: 2031
model year: 2032
model year: 2033
model year: 2034
visualizing results .

In [123]:
# get shps and csvs for caspar

scenario_results = {}

for scenario_number, parameters_dict in bimzec_scenarios.items(): 
# scenario_number = 6
# parameters_dict = bimzec_scenarios[scenario_number]
    print(f'\n\nscenario number: {scenario_number}')
    print(f'parameters: \n{parameters_dict}')

    print('running model ... ')
    model = Model(parameters_dict)
    for i in range(12): 
        print(f'model year: {i+2023}')
        model.step()

    print('getting results ... ')
    results_df, shps_all, emissions_df, emissions_perMaterial_df, tons_byMaterialType, tons_byCircularityType = model.make_results_forCaspar()

    scenario_results[scenario_number] = {
        'results_df': results_df,
        'shps_all': shps_all,
        'emissions_df': emissions_df,
        'emissions_perMaterial_df': emissions_perMaterial_df,
        'tons_byMaterialType': tons_byMaterialType,
        'tons_byCircularityType': tons_byCircularityType
    }



scenario number: 1
parameters: 
{'hub_network': 'none', 'network_type': 'road', 'truck_type': 'diesel', 'biobased_type': 'none', 'modularity_type': 'none', 'circularity_type': 'none', 'model_boundary': 'metropolitan region of amsterdam'}
running model ... 
model year: 2023
model year: 2024
model year: 2025
model year: 2026
model year: 2027
model year: 2028
model year: 2029
model year: 2030
model year: 2031
model year: 2032
model year: 2033
model year: 2034
getting results ... 


scenario number: 2
parameters: 
{'hub_network': 'centralized', 'network_type': 'water', 'truck_type': 'diesel', 'biobased_type': 'none', 'modularity_type': 'none', 'circularity_type': 'none', 'model_boundary': 'metropolitan region of amsterdam'}
running model ... 
model year: 2023
model year: 2024
model year: 2025
model year: 2026
model year: 2027
model year: 2028
model year: 2029
model year: 2030
model year: 2031
model year: 2032
model year: 2033
model year: 2034
getting results ... 


scenario number: 3
par

In [128]:
for scenario_number, results in scenario_results.items(): 
    # print(f'\n\nscenario number: {scenario_number}')
    # print(f'results: \n{results["results_df"]}')
    # print(f'emissions_df: \n{results["emissions_df"]}')
    # print(f'emissions_perMaterial_df: \n{results["emissions_perMaterial_df"]}')
    # print(f'tons_byMaterialType: \n{results["tons_byMaterialType"]}')
    # print(f'tons_byCircularityType: \n{results["tons_byCircularityType"]}')
    # for gdf_name, gdf in results['shps_all'].items(): 
    #     print(f'{gdf_name}: \n{gdf.head()}')

    # save results
    results['results_df'].to_csv(f'results/forCaspar_240209/s{scenario_number}_results.csv', index=False)
    results['emissions_perMaterial_df'].to_csv(f'results/forCaspar_240209/s{scenario_number}_emissions_perMaterial.csv', index=False)
    results['tons_byMaterialType'].to_csv(f'results/forCaspar_240209/s{scenario_number}_tons_byMaterialType.csv', index=False)
    results['tons_byCircularityType'].to_csv(f'results/forCaspar_240209/s{scenario_number}_tons_byCircularityType.csv', index=False)

    # save shps
    for gdf_name, gdf in results['shps_all'].items(): 
        if not gdf.empty:
            gdf.to_file(f'results/forCaspar_240209/s{scenario_number}_{gdf_name}.shp')


C:\Users\tpytsui\AppData\Local\Temp\ipykernel_19344\477910880.py:20: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



In [ ]:
for scenario_number, parameters_dict in bimzec_scenarios.items(): 
    print(f'\n\nscenario number: {scenario_number}')
    print(f'parameters: \n{parameters_dict}')

    print('running model ... ')
    model = Model(parameters_dict)
    for i in range(12): 
        print(f'model year: {i+2023}')
        model.step()

    print('visualizing results ... ')
    emissions_text, fig_emissions, fig_emissions_perMaterial, fig_materials_all, fig_materials_hubs, map_html, results_df = model.visualize()
    fig_dict = {
        'emissions': fig_emissions, 
        'emissions_perMaterial': fig_emissions_perMaterial, 
        'materialComposition_MRA': fig_materials_all, 
        'materialComposition_hubs': fig_materials_hubs, 
        'foliumMap': map_html, 
    }

    print('saving results ... ')
    # save plotly charts 
    import plotly.offline as pyo
    fig_name_list = list(fig_dict.keys())
    fig_name_list.remove('foliumMap')
    for fig_name in fig_name_list: 
        html_file_path = f'results/bimzecScenarios/s{scenario_number}_{fig_name}.html'
        pyo.plot(fig_dict[fig_name], filename=html_file_path, auto_open=False)

    # save folium map 
    with open(f'results/bimzecScenarios/s{scenario_number}_foliumMap.html', 'w', encoding='utf-8') as html_file: 
        html_file.write(fig_dict['foliumMap'])

    # save text file
    with open(f'results/bimzecScenarios/s{scenario_number}_resultsText.txt', 'w', encoding='utf-8') as text_file:
        text_file.write(emissions_text)
        
    # save csv of results
    results_df['scenario'] = scenario_number
    results_df.to_csv(f'results/bimzecScenarios/s{scenario_number}_results.csv', index=False)



scenario number: 1
parameters: 
{'hub_network': 'none', 'network_type': 'road', 'truck_type': 'diesel', 'biobased_type': 'none', 'modularity_type': 'none', 'circularity_type': 'none', 'model_boundary': 'metropolitan region of amsterdam'}
running model ... 
model year: 2023
model year: 2024
model year: 2025
model year: 2026
model year: 2027
model year: 2028
model year: 2029
model year: 2030
model year: 2031
model year: 2032
model year: 2033
model year: 2034
visualizing results ... 
saving results ... 


scenario number: 2
parameters: 
{'hub_network': 'centralized', 'network_type': 'water', 'truck_type': 'diesel', 'biobased_type': 'none', 'modularity_type': 'none', 'circularity_type': 'none', 'model_boundary': 'metropolitan region of amsterdam'}
running model ... 
model year: 2023
model year: 2024
model year: 2025
model year: 2026
model year: 2027
model year: 2028
model year: 2029
model year: 2030
model year: 2031
model year: 2032
model year: 2033
model year: 2034
visualizing results .

In [82]:
results_dfs = []
area = 'whole model'
for i in range(6): 
    scenario_number = i + 1
    results_df = pd.read_csv(f'results/bimzecScenarios/s{scenario_number}_results.csv')
    results_df = results_df[results_df.area == area]
    results_dfs.append(results_df)
    
results_all = pd.concat(results_dfs)
def convertCO2(row): 
    if row.result_name == 'co2': 
        row.value = row.value * 1000000
        row.unit = 'g'
    return row
results_all = results_all.apply(lambda row: convertCO2(row), axis=1)

emissions_names = ['co2', 'NOX', 'PM2.5', 'PM10']
emissions = results_all[results_all.result_name.isin(emissions_names)]
logistics = results_all[results_all.result_name == 'logistic movements']

import plotly.express as px
import plotly.graph_objs as go

# Create the emissions plot with different colors for each emission type
fig = px.line(emissions, x='scenario', y='value', color='result_name', 
              title=f'BAU to BIMZEC for {area}', labels={'value': 'Emissions (grams)'})
fig.update_traces(mode='lines')

# Add the logistics plot as a secondary trace on the right y-axis with dotted line style
logistics_trace = go.Scatter(
    x=logistics['scenario'], y=logistics['value'], 
    mode='lines', name='Logistics', yaxis='y2',
    line=dict(color='black', dash='dot')  # Set line style to dotted
)

fig.add_trace(logistics_trace)

# Update the layout to include a secondary y-axis and move the legend
fig.update_layout(
    width=1000,
    xaxis=dict(title='Scenario'),
    yaxis=dict(title='Emissions (grams)', side='left', showgrid=False),
    yaxis2=dict(title='Logistics (km)', side='right', overlaying='y', showgrid=False),
    legend=dict(title_text='Legend', x=1.1, xanchor='left'),  # Adjust legend position
)

# Show the figure
fig.show(renderer='iframe')
